In [3]:
%pylab inline
import numpy as np
import pandas as pd
import arnie.utils as utils
import arnie.pfunc as pfunc
import seaborn as sns
sns.set_style('white')
sns.set_context('poster')

from matplotlib.colors import LogNorm

from glob import glob
import os, sys, pickle, requests

df = pd.read_csv('ribologic_SI.txt',delimiter='\t')

df = df[df["ligand"] != 'miRNA']

df['log_kd_no_ligand'] = np.log(np.where(df['switch']=='OFF',df['Kd_ON'],df['Kd_OFF'])) - 0.72
df['log_kd_with_ligand'] = np.log(np.where(df['switch']=='ON',df['Kd_ON'],df['Kd_OFF'])) - 0.72

packages=['contrafold_2','vienna_2','rnastructure']

df = df.assign(kd_est_no_lig_contrafold_2=np.nan)
df = df.assign(kd_est_no_lig_vienna_2=np.nan)
df = df.assign(kd_est_with_lig_contrafold_2=np.nan)
df = df.assign(kd_est_with_lig_vienna_2=np.nan)

Populating the interactive namespace from numpy and matplotlib


In [1]:
# Notebook here for reference. Mainly inelegant df handling/plotting

In [ ]:
aptamers = {'FMN':[('nAGGAUAU', '(xxxxxx('),('AGAAGGn', ')xxxxx)')],
            'FMN_rev':[('AGAAGGn', '(xxxxx('),('nAGGAUAU', ')xxxxxx)')],
            'Theophylline':[('GAUACCAG','(xxx(((('),('CCCUUGGCAGC',')xxx)))xxx)')],
            'Theophylline_rev':[('CCCUUGGCAGC','(xxx(((xxx('),('GAUACCAG',')xxx))))')],
            'Tryptophan':[('AGGACCGG','((xxx((('),('CCGCCACU',')))xxx))')],
            'Tryptophan_rev':[('CCGCCACU','(((xxx(('),('AGGACCGG','))xxx)))')]}

concentration = {'FMN':200e-6,'Theophylline':2e-3,'Tryptophan':2.4e-3}
intrinsic_kd = {'FMN':2.2e-6,'Theophylline':20e-6,'Tryptophan':13e-6}

for index, row in df.iterrows():
    if row['ligand'] in ['FMN','Theophylline','Tryptophan']:
        if np.isnan(df.at[index,'kd_est_no_lig_vienna_2']):
            seq = row['sequence']
            ligand = row['ligand']
            ligand_bonus = concentration[ligand]/intrinsic_kd[ligand]
    
            MS2_aptamer = utils.write_constraints(seq, MS2=True)
            lig_aptamer = utils.write_constraints(seq, LIG=True, 
                       lig1 = aptamers[ligand][0],lig2 = aptamers[ligand][1])

            if len(lig_aptamer) > len(seq):
                lig_aptamer = utils.write_constraints(seq, LIG=True,
                lig1 = aptamers['%s_rev' % ligand][0],lig2 = aptamers['%s_rev' % ligand][1])

            MS2_lig_aptamer = utils.write_constraints(seq, MS2=True, LIG=True, 
                       lig1 = aptamers[ligand][0],lig2 = aptamers[ligand][1])

            for pkg in packages:
                Z = pfunc.pfunc(seq, package=pkg)
                Z_MS2 = pfunc.pfunc(seq, package=pkg, constraint=MS2_aptamer)
                Z_lig = pfunc.pfunc(seq, package=pkg, constraint=lig_aptamer)

                try:
                    Z_MS2_lig = pfunc.pfunc(seq, package=pkg, constraint=MS2_lig_aptamer)
                except:
                    Z_MS2_lig = 0

                df.at[index,'kd_est_no_lig_%s' % pkg] = np.log(Z/Z_MS2)
                df.at[index,'kd_est_with_lig_%s' % pkg] =np.log((Z+ligand_bonus*Z_lig)/(Z_MS2+ligand_bonus*Z_MS2_lig))

In [ ]:
df['log_AR']=np.log(df['activation_ratio'])
df['pred_log_AR_vienna_2'] = np.where(df['switch']=='ON',
                                   df['kd_est_no_lig_vienna_2']-df['kd_est_with_lig_vienna_2'],
                                     df['kd_est_with_lig_vienna_2']-df['kd_est_no_lig_vienna_2'])
df['pred_log_AR_contrafold_2'] = np.where(df['switch']=='ON',
                                   df['kd_est_no_lig_contrafold_2']-df['kd_est_with_lig_contrafold_2'],
                                   df['kd_est_with_lig_contrafold_2']-df['kd_est_no_lig_contrafold_2'])

In [ ]:
figure(figsize=(24,12))
x_list=['log_kd_no_ligand','log_kd_with_ligand','log_AR','log_AR',
        'log_kd_no_ligand','log_kd_with_ligand','log_AR','log_AR']
y_list=['kd_est_no_lig_vienna_2','kd_est_with_lig_vienna_2','pred_log_AR_vienna_2','pred_log_AR_vienna_2',
        'kd_est_no_lig_contrafold_2','kd_est_with_lig_contrafold_2','pred_log_AR_contrafold_2','pred_log_AR_contrafold_2']

labels=['Log kd, -ligand','Log kd, +ligand','AR, OFF switches','AR, ON switches','Log kd, -ligand','Log kd, +ligand','AR, OFF switches','AR, ON switches']
ctr=0
for x,y in list(zip(x_list,y_list)):
    subplot(2,4,ctr+1)
    if ctr == 2 or ctr == 6:
        rmse = np.sqrt(np.mean(np.square(np.subtract(df[x][df['switch']=='OFF'],df[y][df['switch']=='OFF']))))

        hexbin(df[x][df['switch']=='OFF'],df[y][df['switch']=='OFF'], gridsize=40,cmap='gist_heat_r',mincnt=1,norm=LogNorm())
        plot([-1,2],[-1,2],linestyle=':',color='k')
    elif ctr == 3 or ctr == 7:
        rmse = np.sqrt(np.mean(np.square(np.subtract(df[x][df['switch']=='ON'],df[y][df['switch']=='ON']))))

        hexbin(df[x][df['switch']=='ON'],df[y][df['switch']=='ON'], gridsize=40,cmap='gist_heat_r',mincnt=1,norm=LogNorm())
        plot([-1,2],[-1,2],linestyle=':',color='k')
    else:
        rmse = np.sqrt(np.mean(np.square(np.subtract(df[x],df[y]))))

        hexbin(df[x],df[y], gridsize=40,cmap='gist_heat_r',mincnt=1,norm=LogNorm(),extent=[0,6,0,6])
        xlim([0,6])
        ylim([0,6])
        plot([0,6],[0,6],linestyle=':',color='k')

    title("%s, RMSE = %.2f" % (labels[ctr],rmse),fontsize=20)
    xlabel('Experiment')
    ylabel('Predicted')
    ctr+=1

tight_layout()